# RBM emulator for coupled channels scattering

We would like to construct an emulator for a toy problem for inelastic nucleon-nucleus scattering in which we explicitly include a few excited states of the target in our model space. We will use a simple model with two excited states, with Woods-Saxon form factors on the diagonal, and Gaussian off the diagonal.

\begin{equation}
 H_{int} = 
 \begin{bmatrix}
   V(r) & \beta g(r) & \gamma g(r) \\
  \beta g(r) & V(r) & \gamma g(r) \\
  \beta g(r) & \gamma g(r) & V(r) \\
 \end{bmatrix},
\end{equation}

where 

\begin{equation}
V(r) = \frac{V+iW}{1 + \exp{ \left( \frac{r-R}{a} \right) }},
\end{equation}

and 

\begin{equation}
g(r) = \exp{ \left( \frac{-r^2}{\alpha^2} \right) }.
\end{equation}

In this toy model, the real part of the Woods-Saxon in the $n$th diagonal represents the interaction of the projectile and target in the $n$th excited state, with $n=0$ being the ground state (and, therefore, the elastic channel). The imaginary component represents the transition from the $n$th channel to other excited states not explicitly included in the model space ($n$ > 2). The off-diagonal terms represent coupling between channels, with $\beta$ controlling the strength of coupling of the ground state to each excited state, and $\gamma$ between excited states.

The Schroedinger-like equation governing scattering in channel $n$ will look like


\begin{equation}
\left( \frac{\hbar^2}{2\mu}\frac{d^2}{dr^2} + \frac{\ell(\ell+1)}{r^2} + V(r) - E_n \right) \psi_n(r) = \sum_{m \neq n} g_{mn}(r),
\end{equation}

where 


\begin{equation}
g_{mn}(r) = \begin{cases} 
    \gamma g(r) & m,n = 0 \\
    \beta g(r) & \text{else}
\end{cases}
\end{equation}

In the case that $\gamma = 0$, this is equivalent to the distorted-wave Born approximation for the excited channels. If we turn on coupling between states with $\gamma \neq 0$, then this is a full coupled-channels calculation.

